# DataProcessing Notebook - AAAG

In [1]:
# Basics Importation 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')

# For Detailed Stats Output
import statsmodels.api as sm
# The linear regression models 
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LassoCV, ElasticNetCV
# Making it so that we can see all columns of the dataframe
pd.set_option('display.max_columns', None)

# Import data
df_train=pd.read_csv('df_train_area_bath_clean.csv',index_col='Id')
df_test=pd.read_csv('df_test_area_bath_clean.csv',index_col='Id')
df_total=pd.read_csv('df_total_area_bath_clean.csv',index_col='Id')

In [2]:
nominal_var_processed=['MSZoning','Street','Alley','LotShape','LandContour','LotConfig','LandSlope',\
             'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',\
             'Exterior1st','Exterior2nd','MasVnrType','Foundation',\
             'BsmtFinType1','Heating','CentralAir',\
             'Electrical','Functional','GarageType','GarageFinish',\
             'PavedDrive','Fence','MiscFeature','SaleType','SaleCondition','MSSubClass',\
             'WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MoSold']
ordinal_var_processed=['OverallQual','OverallCond','BedroomAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars',\
             'YrSold','HeatingQC','KitchenQual','ExterQual','ExterCond','BsmtQual',\
             'BsmtCond','BsmtExposure','FireplaceQu','GarageQual','GarageCond','TotalBaths']
cont_var_processed=['LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea',\
          'GrLivArea','GarageYrBlt','GarageArea','TotalSF']
cont_var_for_tuning=ordinal_var_processed+cont_var_processed

In [3]:
def missingValuesInfo(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100, 2)
    temp = pd.concat([total, percent], axis = 1,keys= ['Total', 'Percent'])
    return temp.loc[(temp['Total'] > 0)]

In [4]:
def feature_standardize(data,scaleType='standardize'):
    '''
    - Accepts a dataframe column
    '''
    if scaleType not in ['standardize', 'normalize']: 
        raise ValueError('%s is not a valid choice' %(scaleType))
    mean_value=np.mean(data)
    standard_dev=np.std(data)
    min_value=np.min(data)
    max_value=np.max(data)
    if scaleType == 'standardize':
        return((data-mean_value)/standard_dev) 
    elif scaleType == 'normalize':
        return((data-min_value)/(max_value-min_value))

In [5]:
def dummify_column(dataframe,column_name):
    '''
    ### NOTE!!! ### vector operation not working yet!!!! ###
    - dataframe takes the entire dataframe you are working on
    - column_name takes a list of strings, where the strings are the column names
    '''
    for feature in column_name:
        dummified_feature = pd.get_dummies(dataframe.loc[:,feature], prefix=feature, prefix_sep='__',drop_first=True)
        dummified_df = pd.concat([dataframe.drop(feature,axis=1),dummified_feature],axis=1,sort='False')
    return dummified_df

In [6]:
def delete_outliers(dataframe,column_name,threshold=5):
    '''
    - Accepts a full dataframe
    - Specify column_name to indicate which columns to check for outliers
    - Column name accepts list of strings, where the strings are the column names
    - threshold is the number of deviations that will be used for removing outliers
    '''
    for feature in column_name:
        standard_dev=np.std(dataframe.loc[:,feature])
        dataframe=dataframe.drop(dataframe[dataframe.loc[:,feature]>standard_dev*threshold].index,axis=0)
    return dataframe


In [7]:
# Check to see if nominal_var is contained in df
nominal_var_processed=['MSZoning','Street','Alley','LotShape','LandContour','LotConfig','LandSlope',\
             'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',\
             'Exterior1st','Exterior2nd','MasVnrType','Foundation',\
             'BsmtFinType1','Heating','CentralAir',\
             'Electrical','Functional','GarageType','GarageFinish',\
             'PavedDrive','Fence','MiscFeature','SaleType','SaleCondition','MSSubClass',\
             'WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MoSold']

######## check for df_train ############################
i=0
for feature in nominal_var_processed:
    if not df_train.loc[:,feature].empty:
        i+=1
print(np.array(nominal_var_processed).size==i)

######## check for df_test ############################
i=0
for feature in nominal_var_processed:
    if not df_test.loc[:,feature].empty:
        i+=1
print(np.array(nominal_var_processed).size==i)

######## check for df_total ############################
i=0
for feature in nominal_var_processed:
    if not df_total.loc[:,feature].empty:
        i+=1
print(np.array(nominal_var_processed).size==i)

True
True
True


In [8]:
# dummify the nominal categorical columns in df_train ############################
df_train_copy=df_train.copy()
for feature in nominal_var_processed:
    df_train_copy=dummify_column(df_train_copy,[feature])
dummified_df_train=df_train_copy

# dummify the nominal categorical columns in df_test ############################
df_test_copy=df_test.copy()
for feature in nominal_var_processed:
    df_test_copy=dummify_column(df_test_copy,[feature])
dummified_df_test=df_test_copy

# dummify the nominal categorical columns in df_total ############################
df_total_copy=df_total.copy()
for feature in nominal_var_processed:
    df_total_copy=dummify_column(df_total_copy,[feature])
dummified_df_total=df_total_copy

# Creating an undummified dataframe for reference, for df_train ##################
undummified_df_train=df_train.copy()

# Creating an undummified dataframe for reference, for df_test ##################
undummified_df_test=df_test.copy()

# Creating an undummified dataframe for reference, for df_total ##################
undummified_df_total=df_total.copy()

print(dummified_df_train.shape)
print(dummified_df_test.shape)
print(dummified_df_total.shape)

print(undummified_df_train.shape)
print(undummified_df_test.shape)
print(undummified_df_total.shape)

print(dummified_df_train.isnull().sum().sum())
print(dummified_df_test.isnull().sum().sum())
print(dummified_df_total.isnull().sum().sum())
print('1459 nan for df_total because of SalePrice absence')

print(undummified_df_train.isnull().sum().sum())
print(undummified_df_test.isnull().sum().sum())
print(undummified_df_total.isnull().sum().sum())
print('1459 nan for df_total because of SalePrice absence')

(1457, 229)
(1459, 218)
(2913, 230)
(1457, 67)
(1459, 66)
(2913, 67)
0
0
1456
1459 nan for df_total because of SalePrice absence
0
0
1456
1459 nan for df_total because of SalePrice absence


In [9]:
# Creating dataframes with no outlier taken out
# This is for maintaining integrity of df_test
with_outlier_dummified_df_train=dummified_df_train.copy()
with_outlier_dummified_df_test=dummified_df_test.copy()
with_outlier_dummified_df_total=dummified_df_total.copy()
with_outlier_undummified_df_train=undummified_df_train.copy()
with_outlier_undummified_df_test=undummified_df_test.copy()
with_outlier_undummified_df_total=undummified_df_total.copy()

In [10]:
standardized_with_outlier_dummified_df_train=with_outlier_dummified_df_train.copy()
standardized_with_outlier_dummified_df_train[cont_var_for_tuning]=standardized_with_outlier_dummified_df_train[cont_var_for_tuning].apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)
standardized_with_outlier_dummified_df_total=with_outlier_dummified_df_total.copy()
standardized_with_outlier_dummified_df_total[cont_var_for_tuning]=standardized_with_outlier_dummified_df_total[cont_var_for_tuning].apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)
standardized_with_outlier_dummified_df_test=with_outlier_dummified_df_test.copy()
standardized_with_outlier_dummified_df_test[cont_var_for_tuning]=standardized_with_outlier_dummified_df_test[cont_var_for_tuning].apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)

##########################################
# Undummified versions for categorical grouping
##########################################

standardized_with_outlier_undummified_df_train=with_outlier_undummified_df_train.copy()
standardized_with_outlier_undummified_df_train[cont_var_for_tuning]=standardized_with_outlier_undummified_df_train[cont_var_for_tuning].apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)
standardized_with_outlier_undummified_df_total=with_outlier_undummified_df_total.copy()
standardized_with_outlier_undummified_df_total[cont_var_for_tuning]=standardized_with_outlier_undummified_df_total[cont_var_for_tuning].apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)
standardized_with_outlier_undummified_df_test=with_outlier_undummified_df_test.copy()
standardized_with_outlier_undummified_df_test[cont_var_for_tuning]=standardized_with_outlier_undummified_df_test[cont_var_for_tuning].apply(lambda x:feature_standardize(x,scaleType='standardize'),axis=0)


In [11]:
intersection_total_train=set(standardized_with_outlier_dummified_df_total.columns)&set(standardized_with_outlier_dummified_df_test.columns)
intersection_cols=set(standardized_with_outlier_dummified_df_train.columns)&intersection_total_train
intersection_cols_with_Sales=list(intersection_cols)+['SalePrice']
# standardized_with_outlier_dummified_df_total[intersection_cols_with_Sales]

In [12]:
standardized_with_outlier_dummified_df_total.loc[:,intersection_cols_with_Sales].to_csv('standardized_dummified_df_total_area_bath_clean.csv')
standardized_with_outlier_dummified_df_test.loc[:,intersection_cols].to_csv('standardized_dummified_df_test_area_bath_clean.csv')
standardized_with_outlier_dummified_df_train.loc[:,intersection_cols_with_Sales].to_csv('standardized_dummified_df_train_area_bath_clean.csv')

standardized_with_outlier_undummified_df_total.loc[:,intersection_cols_with_Sales].to_csv('standardized_undummified_df_total_area_bath_clean.csv')
standardized_with_outlier_undummified_df_test.loc[:,intersection_cols].to_csv('standardized_undummified_df_test_area_bath_clean.csv')
standardized_with_outlier_undummified_df_train.loc[:,intersection_cols_with_Sales].to_csv('standardized_undummified_df_train_area_bath_clean.csv')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [13]:
standardized_with_outlier_dummified_df_total.shape

(2913, 230)